In [1]:
import numpy as np
import os.path
import logging
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import rc
import pickle
from datetime import date
from config import deepqd_config
from cgsp import *

directory =  '.'
config_dict = directory + '/config/20-11-04-04-27-05l32M32N32MCB4000wall-testNN0'
args = deepqd_config()
with open(config_dict,'rb') as f:
    args.__dict__ = pickle.load(f)
if th.cuda.is_available():
    args.device = 'cuda'
    args.num_device = 2
else:
    args.device = 'cpu'
#####
output_dir = './figures/' + args.experiment_name
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    print("Directory " , output_dir ,  " Created ")



In [3]:
Jt = 10
nbatch = 8000
reps = 20
stype='wall'
for depth in [0]:
    deepqd = spectral_tree(args)
    ham = deepqd.hilbert
    exact_sampling = True if ham.Lx<20 else False
    out_appdix = 'exact.png' if exact_sampling else 'mc.png'
    T = ham.Lx * Jt
    try:
        with open('./benchmark/{}Jz-1L32T10.0.tenpy'.format(stype),'rb') as file:
            ref_Jt = pickle.load(file)
            ref_entropy = pickle.load(file)
            ref_szsz_avg = pickle.load(file)
            ref_stagger_avg = pickle.load(file)
            ref_sz_avg = pickle.load(file)
    except:
        print('can not load benchmarking data')

    if depth>0:
        deepqd.load_tree(directory)
        deepqd.load_all_model(directory=directory)
    else:
        deepqd.load_model('0',directory=directory)
    time = np.arange(51) / 50 * T
    entropy_list = []
    nn_coupling_list = []
    spin_imbalance_list = []
    mirror_szsz_list = []
    spin_density = []
    for t in time:
        print('working on t={}, exact={}'.format(t,exact_sampling))
        if exact_sampling:
            entropy, szsz_avg, stagger_avg, sz_avg = deepqd.get_observation(depth,t,exact=True)
        else:
            entropy ,szsz_avg, mirror_szsz_avg, stagger_avg, sz_avg = 0, 0, 0,0, 0
            for q in range(reps):
                _, szsz, mirror_szsz, stagger, sz = deepqd.get_observation(depth=depth,t=t,batchsize=nbatch)
                szsz_avg += szsz / reps
                mirror_szsz_avg += mirror_szsz / reps
                stagger_avg += stagger / reps
                sz_avg += sz / reps
        entropy_list.append(entropy)
        nn_coupling_list.append(szsz_avg)
        mirror_szsz_list.append(mirror_szsz_avg)
        spin_imbalance_list.append(stagger_avg)
        spin_density.append(sz_avg)

    entropy = np.array(entropy_list)
    nn_coupling = np.array(nn_coupling_list)
    mirror_szsz = np.array(mirror_szsz_list)
    spin_imbalance = np.array(spin_imbalance_list)
    spin_density = np.array(spin_density)


can not load benchmarking data
working on t=0.0, exact=False
working on t=6.4, exact=False
working on t=12.8, exact=False
working on t=19.2, exact=False
working on t=25.6, exact=False
working on t=32.0, exact=False
working on t=38.4, exact=False
working on t=44.800000000000004, exact=False
working on t=51.2, exact=False
working on t=57.599999999999994, exact=False
working on t=64.0, exact=False
working on t=70.4, exact=False
working on t=76.8, exact=False
working on t=83.2, exact=False
working on t=89.60000000000001, exact=False
working on t=96.0, exact=False
working on t=102.4, exact=False
working on t=108.80000000000001, exact=False
working on t=115.19999999999999, exact=False
working on t=121.6, exact=False
working on t=128.0, exact=False
working on t=134.4, exact=False
working on t=140.8, exact=False
working on t=147.20000000000002, exact=False
working on t=153.6, exact=False
working on t=160.0, exact=False
working on t=166.4, exact=False
working on t=172.8, exact=False
working on 

In [4]:
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 2.0 #set the value globally

font = {'family': 'serif',
        'color':  'k',
        'weight': 'normal',
        'size': 10,
        }

L = ham.Lx
### <simga_z> at the boundary
idx_list = [0, L//2-1, L//2, -1, L//4-1, L//4, L//4-1+L//2, L//4+L//2]
color_list = ['b','c','k','g','m','r','y','purple']
marker = ['x','.','x','.','x','.','x','.']
for idx, i in  enumerate(idx_list):
    x = args.Jx * np.array(time)
    sparse_filter = (np.arange(x.shape[0]) % 2) == 0
    try:
        ref_spin_density = np.roll(ref_sz_avg,L//4,axis=1)
        plt.plot(ref_Jt, ref_spin_density[:,i], 
                 color=color_list[idx],linewidth=3)
    except:
        print('no benchmarking data')
    plt.scatter( x[sparse_filter], spin_density[:,i][sparse_filter], 
                s = np.ones_like(sparse_filter)*169,
                marker = marker[idx],
                color=color_list[idx], label=r'$s_{'+str((i-8)%32+1)+'}$')
plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel(r'$\langle\sigma_i(t)\rangle$',fontdict=font)
ax = plt.gca()
ax.tick_params(direction='in',labelsize=28)
lgd = plt.legend(fontsize=16,bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plot_dir  = output_dir + "/boundary_sz_l{}T{}".format(depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150,bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.close()


no benchmarking data
no benchmarking data
no benchmarking data
no benchmarking data
no benchmarking data
no benchmarking data
no benchmarking data
no benchmarking data


In [5]:

##### predicted <simga_z>
plt.figure()
ax = plt.gca()
plt.imshow(np.roll(spin_density.transpose(),-8,axis=0), cmap='viridis', 
           interpolation='nearest',
           aspect = 0.25,
           #aspect =spin_density.shape[0]/spin_density.shape[1],
          extent=[0,10,0,L+1])
plt.xlabel(r'$Jt$')
plt.ylabel(r'$\langle\sigma_i^z(t)\rangle$',fontdict=font)
plt.xticks([], [])
plt.yticks([0,10,20,30], [0,10,20,30])
ax.tick_params(direction='in',labelsize=10)
cbar = plt.colorbar()
cbar.ax.tick_params(axis='y',direction='in',labelsize=8)
plot_dir  = output_dir + "/sz_l{}T{}".format(depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150,bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.close()

##### reference <simga_z>
try:
    plt.figure()
    ax = plt.gca()
    plt.imshow(ref_sz_avg.transpose(), cmap='viridis', interpolation='nearest',
              aspect = 0.25,
              extent=[0,10,0,L+1])
    plt.xlabel(r'$Jt$')
    plt.ylabel(r'$\langle\sigma_i^z(t)\rangle$',fontdict=font)
    plt.xticks([], [])
    plt.yticks([0,10,20,30], [0,10,20,30])
    ax.tick_params(direction='in',labelsize=10)
    cbar = plt.colorbar()
    cbar.ax.tick_params(axis='y',direction='in',labelsize=8)
    plot_dir  = output_dir + "/ref_sz_l{}T{}".format(depth,T) + out_appdix
    plt.savefig(plot_dir,dpi=150,bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.close()
except:
    pass

In [6]:

###### mirror correlation
plt.figure()
ax = plt.gca()
szsz = spin_density[:,:L//2] * np.flip(spin_density[:,L//2:],1)
mirror_corr = mirror_szsz-szsz
mirror_corr = (mirror_corr[:,:L//4] + np.flip(mirror_corr[:,L//4:],1))/2
plt.imshow(mirror_corr.transpose(), cmap='gray', interpolation='nearest',
           aspect =1.025,extent=[0,Jt,L//4+0.5,0.5])
plt.xlabel(r'$Jt$',fontdict=font)
plt.ylabel('Corr')

plt.xticks(np.arange(0,11,2), np.arange(0,11,2))
plt.yticks(np.arange(1,9,2), np.arange(8,0,-2))

ax.tick_params(direction='in',labelsize=10)
cbar = plt.colorbar()
cbar.ax.tick_params(axis='y',direction='in',labelsize=8)
    
plot_dir  = output_dir+"/corr_l{}T{}".format(depth,T) + out_appdix
plt.savefig(plot_dir,dpi=150)
plt.close()
